In [1]:
import requests
from bs4 import BeautifulSoup,NavigableString
import random
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import re
from tqdm.asyncio import tqdm
import asyncio
import aiohttp
from ollama import AsyncClient
import ollama

## Scraping

### Find all the urls

In [2]:
url='https://www.wikihow.com/Category:Gardening'
def get_all_pages(url,releted=True):
    base_url='https://www.wikihow.com'
    response = requests.get(url)
    pages=set()
    categories=set()
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cat_sections=soup.find_all('div',class_='cat_section')
        for cat_section in cat_sections:
            if cat_section.get('id')=='cat_featured':
                #print('Feautured')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_all':
                #print('ALL')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_sub_categories' and releted:
                #print('SUBCAT')
                uls=cat_section.find_all('ul')
                for ul in uls:
                    for li in ul.find_all('li'):
                        #print(li)
                        for a in li.find_all('a',class_='cat_link'):
                            categories.add(base_url+a.get('href'))
        return pages,categories

In [3]:
urls,categories=get_all_pages(url,True)


In [4]:
urls2= set()
for category in categories:
    u,c = get_all_pages(category,False)
    urls2.update(u)
    if c:
        print(c)

In [5]:
urls.update(urls2)

In [6]:
len(urls)

998

### Get the page and process with soup

In [7]:
#url = 'https://www.wikihow.com/Prune-a-Tree'
def get_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return {'title': url.split('/')[-1], 'soup':soup, 'url': url }
    else:
        print("Failed to retrieve the page.")

In [8]:
soups=[]
for url in tqdm(list(urls)):
    soup = get_page(url)
    soups.append(soup)

100%|██████████| 998/998 [05:29<00:00,  3.03it/s]


In [18]:
soups[0]

{'title': 'Prune-a-Mini-Rose-Bush',
 'soup': <!DOCTYPE html>
 
 <html class="client-nojs" dir="ltr" lang="en">
 <head>
 <meta charset="utf-8"/>
 <title>6 Easy Ways to Prune a Mini Rose Bush - wikiHow</title>
 <script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
 <script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Prune-a-Mini-Rose-Bush","wgTitle":"Prune a Mini Rose Bush","wgCurRevisionId":32865980,"wgRevisionId":32865980,"wgArticleId":12407041,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Gardening","Home and Garden"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar

In [19]:
class Page:
    def __init__(self, data):
        self.title = data['title']
        self.sections = [Section(section) for section in data['sections']]
        self.url = data['url']

    def display_sections(self):
        return [section.display() for section in self.sections]
    
    def __str__(self):
        output = f'Page Title: {self.title}\nURL: https://www.wikihow.com/{self.title.replace(" ","-")}\nSections:\n'
        for section in self.sections:
            output += str(section) + '\n'
        return output

class Section:
    def __init__(self, section_data):
        self.title = section_data['section_title']
        self.type = section_data['section_type']
        self.steps = []
        for sub_i ,subsection in enumerate(section_data['subsections']):
            #self.subsection.append([])
            for i,step_data in enumerate(subsection,1):
                self.steps.append(Step(step_data, i))

    def display(self):
        return f'({self.type}) {self.title} with {len(self.steps)} steps.'
    
    def __str__(self):
        output = f'- ({self.type}) {self.title}  with {len(self.steps)} steps.\n'
        for step in self.steps:
            output += str(step) + '\n'
        return output

class Step:
    def __init__(self, step_data, index):
        self.index = index
        self.image = step_data['image']
        self.title = step_data.get('title', '')  # Use get() to provide a default value
        self.text = step_data.get('text', '')   # Use get() to provide a default value

    def display(self):
        return f'       Step {self.index}: {self.title}\n            {self.text[:50]}...'
    
    def __str__(self):
        return f'   -- Step {self.index}: {self.title} - {self.text[:50]}...'

In [14]:
def adjust_page(page):
    steps=[]
    for num,section in enumerate(page.sections,1):
        new_step=Step({'title' : section.title,'text' : section.steps[0].title+' '+section.steps[0].text,'image':section.steps[0].image},index=num)
        #print(new_step)
        steps.append(new_step)
    page.sections[0].steps = steps
    page.sections[0].type='No type'
    page.sections[0].title='Steps'
    #page.sections[0].steps = steps
    page.sections = [page.sections[0]]

In [20]:
def process_soup(title,soup,url):
    if title in ['Prune-Nandinas','Kill-Poison-Oak','What-Plant-Am-I','Meaning-of-the-Carnation','Grow-Mint-from-Cuttings']:
        return None
    #sections = soup.find_all('div',class_=lambda x: x and x.startswith('section steps'))
    sections = soup.find_all('div', class_=lambda x: x and (x in ['section steps steps_first sticky hide_step_numbers','section steps sticky hide_step_numbers','section steps before_steps_section sticky', 'section steps steps_first sticky', 'section steps sticky']))# == 'section steps steps_first sticky' or x == 'section steps sticky'))
    
    sections_dir=[]
    if len(sections)==0:
        print('No Section',title)
        return None
    for section in sections:
        h3_tag = section.find_all('h3')
        type = section.find_all('div',class_='altblock')
        section_type = 'No type'
        if type:
            section_type = type[0].text.strip()
        if h3_tag:
            section_title = h3_tag[0].text.strip()
        else:
            h2_tag = section.find_all('h2')
            if h2_tag:
                section_title = h2_tag[0].text.strip()
        
        ols=section.find_all('ol',class_=lambda x: x and x.startswith('steps'))
        if len(ols)>1:
            print('Attention more than one ol',title)
        page=[]
        
        divs=[]
        subsections=[]
        for ol in ols:
            step_num=0
            subsection=[]
            for li in ol.find_all('li', id=lambda x: x and x.startswith('step-id')):
                step={}
                step_num+=1
                img_tag = li.find('img')
                if img_tag and img_tag.get('data-src'):
                    step_image = img_tag['data-src']
                    step['image'] = step_image
                else:
                    # Handle missing image or data-src attribute
                    step['image'] = None
                step_div=li.find('div', class_='step')
                for element in step_div.children:
                    if isinstance(element, NavigableString) and element.strip():
                        #non_tag_text += element.strip() + " "
                        text = step.get('text','')
                        text += element.strip() + " "
                        step['text'] = text
                    # Check if element has the class 'whb'
                    elif element.name and 'whb' in element.get("class", []):
                        step_title = step.get('title','')
                        step_title += element.get_text(strip=True)
                        step['title'] = step_title
                    elif element.name == 'a' and 'external' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'span' and 'frac' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'span' and 'mwe-math-element' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'a' and (not element.get("class") or 'intra-article' in element.get("class")):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'i' and not element.get("class"):
                        #non_tag_text += element.text + " " 
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'b' and not element.get("class"):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'div':
                        #print(f'Div Found in part {section_type} step: {step_num} | Article: {title}')
                        divs.append(element)
                    elif element.name == 'ul':
                        ul_elem=[]
                        for li in element.find_all('li'):
                            plain_text = ''.join(text for text in li.contents if isinstance(text, str)).strip()
                            ul_elem.append(plain_text)
                        step_list = ''
                        for elem in ul_elem:
                            step_list += '\n- ' + elem
                        text = step.get('text','')
                        text += step_list + " "
                        step['text'] = text
                        #step['ul_lists']= step_list
                    # Check if element is another tag
                    elif element.name in ['script', 'sup','br','p','span','table']: #table could be processed
                        pass
                    elif element.name:
                        print(f"Other tag ({element.name}) found in part {section_type} step: {step_num} | Article: {title} |: {element}")
                    else:
                        pass
                if not step.get('title') or len(step['title'])<5:
                    print('Problem: ',title,'in the step',step.get('title'),f'found in part {section_type} step: {step_num}')
                subsection.append(step) 
            subsections.append(subsection)
        sections_dir.append({'section_title' : section_title,'section_type' : section_type, 'subsections':subsections})
    return {'title':title,'sections' : sections_dir, 'url': url}#, divs

In [21]:
pages={}
for soup in tqdm(soups):
    processed_soup=process_soup(soup['title'],soup['soup'], soup['url'])
    if processed_soup:
        page=Page(processed_soup)
        if  page.sections[0].type.startswith('1'):
            adjust_page(page)
    #Page(pages['Plant-an-Avocado-Tree'])
        pages[soup['title']] = page

 14%|█▍        | 144/998 [00:01<00:07, 111.77it/s]

Attention more than one ol Plant-a-Mango-Seed


 21%|██▏       | 214/998 [00:02<00:07, 106.36it/s]

Attention more than one ol Plant-an-Avocado-Tree


 35%|███▍      | 349/998 [00:03<00:06, 107.87it/s]

...

 65%|██████▍   | 647/998 [00:06<00:03, 103.20it/s]

Attention more than one ol Grow-Edible-Mushrooms
Attention more than one ol Sharpen-Lawn-Mower-Blades


 69%|██████▉   | 691/998 [00:06<00:03, 100.09it/s]

Problem:  Know-How-Long-It-Takes-for-a-Tree-to-Grow in the step Seed found in part  step: 1
Attention more than one ol Grow-Avocados


100%|██████████| 998/998 [00:09<00:00, 103.08it/s]


In [23]:
def all_one(page):    
    for section in page.sections:
        if len(section.steps)>1:
            return False
    return True

### Tests

In [24]:
c=0
types=set()
for page in pages:
    #if pages[page].sections[0].type == 'No section':
    types.add( pages[page].sections[0].type)
    #if  not pages[page].sections[0].type:#.startswith('No'):
    if  pages[page].sections[0].type.startswith('P'):
        print(pages[page])
        c+=1
        #adjust_page(pages[page])
        #print(pages[page])
print(c,'/',len(pages))

Page Title: Harvest-Celery
URL: https://www.wikihow.com/Harvest-Celery
Sections:
- (Part 1) Knowing When to Harvest  with 5 steps.
   -- Step 1: Wait 3 to 5 months after planting the celery to harvest it. - The growing period for celery is much longer than ...
   -- Step 2: Harvest before the outdoor temperature reaches 65 °F (18 °C). - If you wait until the weather becomes too hot, the...
   -- Step 3: Make sure the lower stalks are at least 6 in (15 cm) long. - Even the smallest stalks on the plant need to be a...
   -- Step 4: Ensure the plant is 3 in (7.6 cm) across to harvest all of the stalks. - To harvest a whole plant, you need to wait until i...
   -- Step 5: Look for stalks that are compact if you want to harvest the whole plant. - It’s best to harvest whole plants that haven’t had...

- (Part 2) Harvesting the Celery  with 4 steps.
   -- Step 1: Harvest the longest stalks on the outside of the plant as needed. - When choosing stalks to remove, look around the ou...
   -- Ste

In [44]:
page_t=list(pages.keys() )[random.randint(0,len(pages))]
print(pages[page_t])#['sections'][0]['steps'][0]

Page Title: Make-Rose-Petal-Beads
URL: https://www.wikihow.com/Make-Rose-Petal-Beads
Sections:
- (Method 1) Plain Rose Petal Beads  with 10 steps.
   -- Step 1: Gather red rose petals. - ...
   -- Step 2: Place on a rusty tin sheet in the sun for a day or so to begin to dry. - Alternatively, if you are familiar with oven or mi...
   -- Step 3: Wait for the petals to shrink before proceeding but not dry. - There needs to be a little moisture left to help w...
   -- Step 4: Put the petals through a mincer or through a blender until there is a mince of roses that appears ideal for forming into beads. - You can test this after each mince or blend by rol...
   -- Step 5: Using your hands, roll small bead shapes with a bit of the mixture. - Do this firmly so that the little bead balls stick...
   -- Step 6: Pierce a hole through the centre of each bead. - Use a long hat-pin, a thin skewer or a piece of wi...
   -- Step 7: Allow the beads to dry thoroughly. - Place in a warm, dry and well-ven

## Create Options and Questions

### Ollama Options generation

In [51]:
client_ollama = AsyncClient(
  host='http://192.168.1.5:11434',
  headers={'x-some-header': 'some-value'}
)

async def chat(content):
    message = {'role': 'user', 'content': content}
    response = await client_ollama.chat(model='llama3.3', messages=[message])
    return response['message']['content']



In [54]:
def add_difficulty(level):
    if level == 1: # Beginner / Very easy difficulty
        prompt = "The distractors should be about a completely different topic, and represent steps from a different step-by-step guide, making them easily distinguishable from the correct answer."
    elif level == 2: # Novice / Easy difficulty
        prompt = "The distractors should be related to the same topic but significantly different from the correct answer. A basic understanding of the topic is sufficient to choose the right option."
    elif level == 3: # Intermediate difficulty
        prompt = "The distractors should be similar to the correct answer but with some notable differences. A moderate understanding of the topic is required to choose the right option."
    elif level == 4: # Advanced difficulty
        prompt = "The distractors should be subtly different from the correct answer, requiring a deep understanding of the topic to perceive these differences."
    elif level == 5: # Expert / Very hard difficulty
        prompt = "The distractors should be close to the correct answer but conceptually and technically different. Only an expert in this field can easily distinguish between them."
    else:
        return 'Invalid difficulty level. Please enter a number between 0 and 4.'
    return prompt

def find_options(string):
    options_start = string.find("Options =")
    if options_start != -1:
        start_bracket = string.find("[", options_start)
        end_bracket = string.find("]", options_start)
        if start_bracket != -1 and end_bracket != -1:
            options=''
            try:
                options = list(eval(string[start_bracket + 1:end_bracket]))
            except:
                print(string)
            if options and not options[0].startswith('distractor'):
                    return options
            else:
                print('Distractor',string)
            
    else:
        print('Response: ',string)
        return None

async def generate_distractors(question,correct_answer,difficulty=0):
    prompt='''Given the following question: {question}.\n\nAnd the correct answer: {correct_answer}.\n\nThink of appropriate distractors for a quiz. 
Provide only 3 plausible distractors. '''.format(question=question,correct_answer=correct_answer)+add_difficulty(difficulty)+'''\nAfter considering the question, list the distractors obtained at the end of your response in the form: Options = ["<distractor_1>", "<distractor_2>", "<distractor_3>"], where "distractor_i" are the distractors generated.'''
    #print(prompt)
    #response= await chat(prompt.format(question=question,correct_answer=correct_answer))
    response=await chat(prompt)
    options=find_options(response)
    return options

async def generate_options(question,correct_answer ,difficulty=0):
    options = await  generate_distractors(question,correct_answer,difficulty)
    if not options:
        return None
    options=options[:3]
    options.append(correct_answer)
    random.shuffle(options)
    letters = ['A','B','C','D']
    options = '\n'.join([f'{letters[i]}. {step}' for i, step in enumerate(options)])
    return options

In [55]:
q='What?'
c_a='Yes'
await generate_distractors(q,c_a,5)

['Affirmative', 'Certainly', 'Indeed']

### Generate Questions

In [72]:
question_templates = {
    'missing_step' : 'What step is missing from the "{section_title}" instructions in the "{page_title}" procedure?\n\nSteps:\n{list_of_steps}',
    
    'next_step': 'What is the next step in the "{section_title}" instructions in the "{page_title}" procedure?\n\nSteps:\n{list_of_steps}.', # No option # Option with LLM
    'open_question': 'Write down all the steps from the "{section_title}" instructions in the "{page_title}" procedure.',

    'rearrange': 'Put these steps in the correct order for the "{section_title}" instructions in the "{page_title}" guide.\n\nSteps: {list_of_steps}.', # No LLM
}

async def create_question(page, page_section,  question_type, options_difficulty=1):
    question = question_templates[question_type]
    list_of_steps = [step.title for step in page_section.steps] # Extracting titles of steps
    if len(list_of_steps)<3: # If there are less than three steps, we can't create a question
        return 'Few steps'
    if question_type in ['missing_step']:
        if len(page_section.steps) > 4:
            missing_step_num = random.randint(3,len(page_section.steps)-2)
        else:
            missing_step_num = random.randint(0,len(page_section.steps)-1)
        #correct_answer = page['steps'][missing_step_num]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "."*len(correct_answer)
    elif question_type == 'next_step':
        correct_answer = list_of_steps[-1]
        list_of_steps.pop()
    elif question_type == 'rearrange':
        correct_answer = list_of_steps[:]
        correct_answer = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(correct_answer)])
        random.shuffle(list_of_steps)
    elif question_type == 'open_question':
        correct_answer = list_of_steps[:]
        correct_answer = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(correct_answer)])
    list_of_steps = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(list_of_steps)])
    options=None
    if question_type in ['missing_step','next_step']:
        options = await generate_options(question,correct_answer,options_difficulty) # difficulty from 0 to 4
        if not options:
            return None
    else:
        options_difficulty=None
    question=question.format(page_title = page.title.replace('-',' '), section_title = page_section.title , list_of_steps = list_of_steps)
    return {'question' : question, 'options' : options,'correct_answer' : correct_answer, 'options_difficulty': options_difficulty, 'question_type': question_type, 'page_title': page.title.replace('-',' '), 'section_title' : page_section.title, 'url' : page.url}


In [73]:
tasks=[]
question_types=["missing_step", "next_step", "open_question", "rearrange"]
sub_pages=list(pages)
for key in tqdm(sub_pages):
    page=pages[key]
    for section in page.sections:
            question_type = random.choice(question_types)
        #for question_type in ["missing_step", "next_step", "open_question", "rearrange"]:
            difficulty=random.randint(1,5)
            if 3<len(section.steps) and question_type=="next_step":
                tasks.append(create_question(page ,section, question_type,difficulty))
            elif 3<len(section.steps)<10: # Check if the section has too much steps to create a question
                tasks.append(create_question(page, section, question_type,difficulty))
print(len(tasks))
            # Generate questions of this type for the current section here


100%|██████████| 993/993 [00:00<00:00, 77805.79it/s]

2155


In [ ]:
results = []
c=0
for task in tqdm(tasks):
    result = await task
    if result is not None:
        results.append(result)
    else:
        print("Result is: ",result)
        c+=1
print(c)

  0%|          | 7/2155 [01:37<8:33:51, 14.35s/it] 

To create plausible distractors for the given correct answer, "Repeat the brining process until your olives are at your desired taste," we need to consider the context of brining olives and the steps involved in such a procedure. The distractors should be similar enough to seem plausible but incorrect or incomplete in a way that requires moderate understanding of the topic to distinguish them from the correct answer.

1. **Incorrect Frequency**: A distractor could suggest repeating the process a fixed number of times rather than until the olives reach the desired taste. This would imply a lack of understanding of how taste preferences can vary and the importance of checking the olives' flavor during the brining process.

2. **Inaccurate Condition for Stopping**: Another distractor might suggest stopping the brining process based on a different criterion, such as time elapsed or the appearance of the olives, rather than their taste. This could lead to olives that are not sufficiently fl

  2%|▏         | 35/2155 [06:15<4:33:31,  7.74s/it]

### Testing

In [562]:
def print_pretty(data):
    # Extract question, options, correct answer, and options difficulty from the data.
    question = data['question']
    options = data['options'].split('\n')  # Split the options into individual lines.
    correct_answer = data['correct_answer']
    options_difficulty = data['options_difficulty']

    print("Question:")
    print(question)
    #print(data['options'])
    if data['options']:
        print("\nOptions:")
        for option in options:  # Print each option on a new line.
            print(option)
    print("\nCorrect Answer:")
    print(correct_answer)
    if data['options']:
        print("\nOptions Difficulty Level (1 to 5):",options_difficulty)

In [575]:
num=random.randint(1,len(results))
print_pretty(results[num])

Question:
What step is missing from the "Making a Large Watering Can" instructions in the "Prune Lilacs" procedure?

Steps:
1. Choose a large bottle with a handle and a screw-on lid.
2. Clean out the bottle and remove any labels.
3. Punch holes into the cap with a nail.
4. Consider adding a hole above the handle.
5. ...........................
6. Use the watering can.

Options:
A. Fill the bottle with water.
B. Measure the correct amount of water to fill the bottle.
C. Clean the bottle before filling it with water.
D. Turn off the tap before filling the bottle with water.

Correct Answer:
Fill the bottle with water.

Options Difficulty Level (1 to 5): 2


In [1]:
types={'open_question':0,'rearrange':0, 'next_step':0,'missing_step':0}
difficulties={1:0,2:0,3:0,4:0,5:0,None:0}
for result in results:
    types[result['question_type']]+=1
    difficulties[result['options_difficulty']]+=1
print(types)
print('Options difficulty',difficulties)

NameError: name 'results' is not defined

### Saving Results

In [600]:
import json

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
save_to_json(results,'wikihow.json')